In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pathlib import Path


In [7]:
#OMXC25 Data
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.79 Safari/537.36'
}

page = 'https://finance.yahoo.com/quote/%5EOMXC25/history?period1=1483228800&period2=1691798400&interval=1mo&filter=history&frequency=1mo&includeAdjustedClose=true'
pageTree = requests.get(page, headers=headers)
pageSoup = BeautifulSoup(pageTree.content, 'lxml')
pageSoup
table_node = pageSoup.find('table',class_="W(100%) M(0)")


def parse_html_table(table_node):
    # Get the columns in a list
    columns_html = table_node.thead.find_all('th')
    # Extract the text
    columns = [col.text for col in columns_html]

    rows_list = table_node.tbody.find_all('tr')

    data = []
    for row_node in rows_list:
        row = []
        for child in row_node.children:
            row.append(child.text)
        data.append(row)
    df = pd.DataFrame(data,columns=columns)
    return df

x = parse_html_table(table_node)
x = x[['Date','Open']]
x['Date'] = pd.to_datetime(x['Date'])
x = x.rename(columns={'Open': 'OMXC25'})
#x.to_csv(Path.cwd() / "OMXC25Data")
x

,Date,OMXC25
0,2023-08-11,"1,795.66"
1,2023-08-01,"1,806.76"
2,2023-07-01,"1,821.58"
3,2023-06-01,"1,813.22"
4,2023-05-01,"1,829.29"
...,...,...
76,2017-05-01,"1,126.25"
77,2017-04-01,"1,075.07"
78,2017-03-01,"1,060.60"
79,2017-02-01,"1,049.63"


In [13]:
#Storebaelt Data
df_storebaelt=pd.read_csv(Path.cwd() / "Report (1).csv", sep=';', header=1)

df_storebaelt['Date'] = pd.to_datetime(df_storebaelt['År'].astype(str) + '-' + df_storebaelt['Måned'].astype(str), format='%Y-%m')
df_storebaelt = df_storebaelt.drop(['År', 'Måned'], axis=1)

df_storebaelt['Storebaelt LastbilerTotal'] = df_storebaelt['Små lastbiler']+df_storebaelt['Store lastbiler']+df_storebaelt['Lastbiler over 20 meter']
df_storebaelt = df_storebaelt.drop(df_storebaelt.columns[:8],axis=1)
df_storebaelt['Storebaelt LastbilerTotal'] = (df_storebaelt['Storebaelt LastbilerTotal'] * 1000).astype(int)  # Multiply by 1000 to shift decimal places

#df_storebaelt.to_csv(Path.cwd() / "data/FinalStorebaeltData")
df_storebaelt

,Date,Storebaelt LastbilerTotal
0,2023-07-01,138135
1,2023-06-01,155207
2,2023-05-01,145951
3,2023-04-01,124993
4,2023-03-01,145877
5,2023-02-01,123946
6,2023-01-01,127485
7,2022-12-01,123894
8,2022-11-01,141909
9,2022-10-01,138921


In [6]:
df_oresund=pd.read_excel(Path.cwd() / "Oresundsbroen.xlsx", header=1)
df_oresund['Date'] = pd.to_datetime(df_oresund['År'].astype(str) + '-' + df_oresund['Måned'].astype(str), format='%Y-%m')
df_oresund = df_oresund.drop(columns=['År', 'Måned','MC', 'Personbiler', 'Personbiler med slæb m.m.', 'Busser', 'Total'])
df_oresund = df_oresund.rename(columns={'Last- og varebiler fra 6 m': 'Oresund LastbilerTotal'})

#df_oresund.to_csv(Path.cwd() / "FinalOresundtData")
df_oresund

,Oresund LastbilerTotal,Date
0,60599,2023-06-01
1,60156,2023-05-01
2,52003,2023-04-01
3,60971,2023-03-01
4,52304,2023-02-01
...,...,...
271,11821,2000-11-01
272,11566,2000-10-01
273,10924,2000-09-01
274,10360,2000-08-01


In [17]:
merged_df = pd.merge(x,df_storebaelt, on='Date')

merged_df = pd.merge(merged_df,df_oresund, on='Date')
merged_df

,Date,OMXC25,Storebaelt LastbilerTotal,Oresund LastbilerTotal
0,2023-06-01,"1,813.22",155207,60599
1,2023-05-01,"1,829.29",145951,60156
2,2023-04-01,"1,789.75",124993,52003
3,2023-03-01,"1,779.34",145877,60971
4,2023-02-01,"1,708.03",123946,52304
5,2023-01-01,"1,708.43",127485,54919
6,2022-12-01,"1,704.52",123894,53085
7,2022-11-01,"1,582.77",141909,57478
8,2022-10-01,"1,429.19",138921,57514
9,2022-09-01,"1,629.48",147674,62845
